<a href="https://colab.research.google.com/github/fritzmartin003/RAG-System-Projekt/blob/main/Hugging_Face_Sara_FineTuning_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notwendige Bibliotheken installieren
!pip install faiss-cpu transformers sentence-transformers pymupdf numpy scipy


In [ ]:
# Imports
import fitz  # PyMuPDF
import numpy as np
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer

In [17]:
from google.colab import userdata
HuggingFaceAPIKey = "HF2"

In [3]:
# PDF-Text extrahieren
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

pdf_path = "SakowskiBuch.pdf"
pdf_text = extract_text_from_pdf(pdf_path)


In [4]:
# Text in Chunks teilen
def split_text(text, chunk_size=300, overlap=90):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = split_text(pdf_text)
print(f" PDF in {len(chunks)} Chunks unterteilt!")


 PDF in 2969 Chunks unterteilt!


In [12]:
import multiprocessing as mp

In [18]:
# Modell laden
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Embeddings berechnen mit batch_size=64
chunk_embeddings = np.array(embedding_model.encode(chunks, batch_size=64, convert_to_numpy=True))

# Ausgabe der Anzahl der berechneten Embeddings
print(f"Anzahl der berechneten Embeddings: {len(chunk_embeddings)}")

# Bestätigung, ob der Prozess geklappt hat
if len(chunk_embeddings) == len(chunks):
    print("Embeddings wurden erfolgreich berechnet.")
else:
    print("Fehler: Die Anzahl der berechneten Embeddings stimmt nicht mit der Anzahl der Chunks überein.")


In [19]:
# FAISS Vektordatenbank aufsetzen
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)
print(" FAISS Vektordatenbank erstellt!")


 FAISS Vektordatenbank erstellt!


In [20]:
# Ähnlichkeitssuche in FAISS
def get_relevant_chunks(query, top_k=3):
    query_embedding = get_embedding(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]


In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os
import torch

# Model und Tokenizer laden
model_name = "mistralai/Mistral-7B-v0.1"
pipe = pipeline("text-generation", model=model_name, device=0 if torch.cuda.is_available() else -1)  # GPU falls verfügbar
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Funktion zur Beantwortung der Frage
def generate_answer(query, temperature=0.7, max_length=256, top_k=3):
    # Relevante Chunks holen (dein Code für Chunk-Auswahl)
    relevant_chunks = get_relevant_chunks(query, top_k=top_k)
    context = "\n".join(relevant_chunks)  # Kontext für die Antwort

    # Prompt formatieren
    prompt = f"Beantworte die Frage basierend auf diesem Kontext:\n\n{context}\n\nFrage: {query}\nAntwort:"

    # Antwort generieren (Sampling oder deterministisch)
    response = pipe(prompt,
                    max_new_tokens=max_length,
                    temperature=temperature,
                    do_sample=True,  # Sampling aktiviert für zufällige Variation
                    top_k=50,        # Top-K Sampling für mehr Vielfalt
                    num_return_sequences=1  # Nur eine Antwort zurück
                   )
    return response[0]["generated_text"]



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
# Test
frage = "Welche Schutzrechte gibt es für werdende Mütter im Arbeitsrecht?"
antwort = generate_answer(frage)
print("Antwort:", antwort)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 